In [157]:
import numpy as np
import pandas as pd
import spiceypy as spice
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.constants import c
from astropy.time import Time
from astroquery.jplhorizons import Horizons

In [83]:
def Furnisher():
    '''
    This function is used to load all kernels needed in an operation.
    Comment out kernels not in use and add the ones in use.
    
    Arguments: NA
    Returns: NA
    
    '''
    spice.kclear()
    spice.furnsh('/Users/user/Downloads/naif0009.tls.txt')
    #spice.furnsh('/Users/user/Downloads/cas00084.tsc.txt')
    #spice.furnsh('/Users/user/Downloads/cpck05Mar2004.tpc.txt')
    #spice.furnsh('/Users/user/Downloads/020514_SE_SAT105.bsp')
    #spice.furnsh('/Users/user/Downloads/981005_PLTEPH-DE405S.bsp')
    #spice.furnsh('/Users/user/Downloads/030201AP_SK_SM546_T45.bsp')
    #spice.furnsh('/Users/user/Downloads/04135_04171pc_psiv2.bc')
    #spice.furnsh('/Users/user/Downloads/cas_v37.tf.txt')
    #spice.furnsh('/Users/user/Downloads/cas_iss_v09.ti.txt')
    spice.furnsh('/Users/user/Downloads/jup310.bsp')
    spice.furnsh('/Users/user/Downloads/jup341.bsp')
    pass

Relevant kernels for jupiter.

In [84]:
Furnisher()

In [54]:
def get_spice_function(name,cor,loc):
    """
    This wrapper function automates the creation of objects through the JPL Horizons database. 
    
    Arguments:
    
    name: str
    
    Stipulates the target object in Horizons. The major bodies in the Solar System have an id based on their position.
    Hence '5' refers to Jupiter and '3' to Earth. A single number designator refers to a barycenter and a designator
    such as '599' to the planetary center. For minor bodies in the Solar System, the id_type in the Horizons
    must be changed to "minorbody"
    
    cor: str
    
    Refers to the type of correction that the object has. Available arguments are 'NONE', 'LT','LT+S'
    
    loc: str
    
    Designates the location of observation. Names that start with "g@#" refer to barycenters where the number designates the 
    body that the observer is based at. Hence "g@0" refers to the Solar System barycenter. Also takes Earth location designators.
    Observatories are named after their code. Hence, Pan-Starrs observatory is referred as "f51"

    Returns:
    
    get_target_xyz function
    """    
    def get_target_xyz(t):
        """
        Returns the vectors of the Horizons body at a certain time t.
        
        Arguments:
        
        t: days
        
        Julian date of observation
        
        Returns:
    
        xyz: numpy array
        
        A position vector of the observed object
    
        uvw: numpy array
        
        An instantaneous velocity vector of the observed object
        
        radec: numpy array
        
        The right ascension and declination of the observed object
        """
        
        state,lighttime = spice.spkezr(name,t,'J2000',cor,loc)
        pos,lighttime = spice.spkpos(name,t,'J2000',cor,loc)
        range,ra,dec = spice.recrad(pos) 
        xyz = np.array([state[0],state[1],state[2]])*6.68459e-9
        uvw = np.array([state[3],state[4],state[5]])*6.68459e-9
        radec = np.array([ra,dec])
        return xyz,uvw,radec*180/np.pi
    return get_target_xyz

In [96]:
def get_astroquery_function(name,cor,loc):
    """
    This wrapper function automates the creation of objects through the JPL Horizons database. 
    
    Arguments:
    
    name: str
    
    Stipulates the target object in Horizons. The major bodies in the Solar System have an id based on their position.
    Hence '5' refers to Jupiter and '3' to Earth. A single number designator refers to a barycenter and a designator
    such as '599' to the planetary center. For minor bodies in the Solar System, the id_type in the Horizons
    must be changed to "minorbody"
    
    cor: str
    
    Refers to the type of correction that the object has. Available arguments are "geometric","astrometric" and 
    "apparent"
    
    loc: str
    
    Designates the location of observation. Names that start with "g@#" refer to barycenters where the number designates the 
    body that the observer is based at. Hence "g@0" refers to the Solar System barycenter. Also takes Earth location designators.
    Observatories are named after their code. Hence, Pan-Starrs observatory is referred as "f51"

    Returns:
    
    get_target_xyz function
    """    
    def get_target_xyz(t):
        """
        Returns the vectors of the Horizons body at a certain time t.
        
        Arguments:
        
        t: days
        
        Julian date of observation
        
        Returns:
    
        xyz: numpy array
        
        A position vector of the observed object
    
        uvw: numpy array
        
        An instantaneous velocity vector of the observed object
        """
        
        obj = Horizons(id = name, location = loc, epochs = t, id_type = 'majorbody')
        obj1 = obj.vectors(aberrations = cor, refplane = 'earth')
        xyz = np.array([float(obj1['x']),float(obj1['y']),float(obj1['z'])])
        uvw = np.array([float(obj1['vx']),float(obj1['vy']),float(obj1['vz'])])
        obj2 = obj.ephemerides(refsystem = 'J2000', extra_precision = True)
        radec = np.array([float(obj2['RA']),float(obj2['DEC']),float(obj1['range'])])
        return xyz,uvw,radec
    return get_target_xyz  

In [127]:
def monthToNum(shortMonth):
    
    '''
    
    Returns the name of a month that a number corresponds to and vice versa
    
    Arguments:
    
    shortMonth: str
    
    Either the first three letters of a month or its number (e.g. 01 for January, 10 for October)
    
    Returns:
    
    shortMonth: str
    
    Either the first three letters of a month or its number (e.g. 01 for January, 10 for October),
    the other one to the one that was given as an argument
    
    '''
    
    return{
        'Jan' : '01', 'Feb' : '02','Mar' : '03','Apr' : '04','May' : '05','Jun' : '06','Jul' : '07','Aug' : '08',
        'Sep' : '09', 'Oct' : '10','Nov' : '11','Dec' : '12','01' : 'Jan','02' : 'Feb','03' : 'Mar','04' : 'Apr',
        '05' : 'May','06' : 'Jun','07' : 'Jul','08' : 'Aug','09' : 'Sep', '10' : 'Oct','11' : 'Nov','12' : 'Dec'}[shortMonth]

In [128]:
def JD_to_SpiceJD(jd):
    '''
    
    Converts jd date to spice units of time.
    
    Arguments:
    
    jd: float
    
    A jd time in days
    
    Returns:
    
    newjd: float
    
    A spice usable time in seconds
    
    
    '''
    t = Time(jd, format='jd', scale='utc')
    k = t.fits
    year = k[0:4]
    month = k[5:7]
    day = k[8:10]
    hours = k[11:13]
    minutes = k[14:16]
    seconds = k[17:23]
    newmnth = monthToNum(month)
    utc = newmnth + ' ' + day + ", " + year
    newjd = spice.str2et(utc) + int(hours)*3600 + int(minutes)*60 + float(seconds)
    return newjd

In [146]:
def Finderf51(name,t):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    get_pans = get_astroquery_function('0','astrometric','f51')
    vecs2 = get_pans(t)
    ts = JD_to_SpiceJD(t)
    get_obj = get_spice_function(name,"LT","SOLAR SYSTEM BARYCENTER")
    vecs = get_obj(ts)
    vectors = vecs2[0]+vecs[0]
    xyz = vectors/6.68459e-9
    xyz = np.array([xyz[0],xyz[1],(xyz[2]-np.sin(20.707222)*4.25875e-5)])
    range, ra, dec = spice.recrad(xyz)
    return np.array([ra,dec])*180/np.pi

Returning radecs of Jovian satellites. The time selected is one of the images of Jupiter that we asked from
Matt. ID 2213. 

Image centre: 

RA: 329.686524
DEC: -13.076262

In [147]:
Finderf51("Jupiter",2454999.077545891)

array([329.39768642, -13.36966214])

In [148]:
Finderf51("Callisto",2454999.077545891)

array([329.54770394, -13.31095003])

In [149]:
Finderf51("Himalia",2454999.077545891)

array([330.15732399, -13.3434753 ])

In [150]:
Finderf51("Pasiphae",2454999.077545891)

array([329.73517058, -12.98779907])

In [151]:
Finderf51("Ananke",2454999.077545891)

array([328.49496908, -14.44858893])

As expected the irregulars are further away from Jupiter than Callisto in the image 

In [145]:
obj = Horizons(id = '504', location = 'f51', epochs = 2454999.077545891, id_type = 'majorbody')
obj.ephemerides()

targetname,datetime_str,datetime_jd,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
---,---,d,---,---,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,---,---,mag,mag,mag / arcsec2,%,arcsec,arcsec,---,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,---,deg,deg,%,deg,deg,deg,deg,---,s,deg,deg,deg,deg,deg,deg,---,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,---,deg,deg,deg
str14,str24,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64
Callisto (504),2009-Jun-16 13:51:39.965,2454999.077545891,,m,329.54791,-13.31089,329.68196,-13.26379,-2.2212,-1.51446,158.2086,53.687,820.63,313.37,525.155,211.567,68.077,21.108546019,1.24,0.161,6.12,6.7,99.212,0.0117,566.2286,*,1.479832,98.94,0.51,109.12,0.07,70.38,0.13,337.9471,0.74,316.8467,-0.7659,5.054053163057,-8.2199146,4.4914691660322,-32.1848699,37.35441,13.38191,32.27678,118.3741,/L,10.191,36.8,43.1,79.2334,250.413,251.551,-0.22678,Cap,66.184517,327.1722704,-0.8629527,268.18601,64.70811,42.956899,-47.323582,3.4317755574,0.000355,--,--,--,--,--,--,--,--,--,--,--,--,--,-0.87025138,10.1882,321.9391,-0.8174


For reference Callisto has about the same radecs in Horizons (within a few pixels)